In [6]:
import json
from googleapiclient.discovery import build

In [7]:
# from llama_index.readers.google import GoogleDriveReader

In [8]:
from langchain_google_community import GoogleDriveLoader

In [10]:
# retrieve all documents from the folder
folder_id = '11eJrf2XbhQTdkgq5J64GtT8NJLAUhvGR'
file_ids = ['1g50DGceqnw0dWaNXlOynNy9OQdcgSZP2']

loader = GoogleDriveLoader(
    # folder_id=folder_id,
    file_ids=file_ids,

    # file_types=["document", "sheet"],
    # file_types=["document"],

    # token generated separately in exp1.ipynb
    token_path="./token.json",

    # Optional: configure whether to recursively fetch files from subfolders. Defaults to False.
    recursive=False,
)

creds = loader._load_credentials()
service = build("drive", "v3", credentials=creds)

doc_ids = loader._fetch_files_recursive(folder_id=folder_id, service=service)

# docs = loader.load()
# docs[0]

In [11]:
len(doc_ids)

154

In [ ]:
doc_id = docs[0].metadata['source'].split('/')[-2]
doc_id

In [ ]:
# retrieve one document from the folder
doc_loader = GoogleDriveLoader(
    document_ids=[doc_id],
    token_path="./token.json",
)
docs = doc_loader.load()

In [ ]:
# The below is not necessary -- just need to run queries against a single plain doc...

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000, chunk_overlap=0, separators=[" ", ",", "\n"]
)
texts = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(texts, embeddings)
retriever = db.as_retriever()

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
qa = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
def get_answer(query: str):
    answer = qa.invoke({"input": query})
    return answer['answer']

def print_answer(query):
    answer = get_answer(query)
    print('ANSWER:', answer)
    return answer

To extract:
* seller name
* seller email address
* buyer name
* buyer email address
* start date
* end date
* total price
* terms months
* product/service names
* price per user per service per month
* number of users per service

In [ ]:
answer = print_answer("There are two companies mentioned. Which company is the seller and which is the buyer? Repond in JSON format with keys 'buyer' and 'seller'")
company = json.loads(answer)
seller, buyer = company['seller'], company['buyer']

In [ ]:
contact_template = "What is the name and email address of the person at company '{}'? Respond in JSON format with 'name' and 'email' being keys."
contact_seller = contact_template.format(seller)
contact_buyer = contact_template.format(buyer)

In [ ]:
print_answer(contact_seller)

In [ ]:
print_answer(contact_buyer)

In [ ]:
print_answer("What is the contract start and end dates? Respond in JSON format with 'start_date' and 'end_date' being keys.")

In [ ]:
print_answer(f"What are the products being sold by {seller} to {buyer}? Respond in JSON format as a list of key-value pairs with the keys 'product_name', 'price_per_user_per_month', 'number_of_users'.")

In [ ]:
print_answer(f"What are the products being sold by {seller} to {buyer}? Respond in JSON format as a list of key-value pairs with the keys 'servicet_name', 'price_per_user_per_month', 'number_of_users'.")